In [4]:
import sys
sys.path.append('../..')


In [65]:
from importlib import reload
import numpy as np
import lib.annotation_api
reload(lib.annotation_api)

<module 'lib.annotation_api' from '../../lib/annotation_api/__init__.py'>

In [64]:
from lib.annotation_api import Api
api = Api('scraper_test_dev')

In [7]:
from lib.obo import read_ontology
to = read_ontology('../../data/geo-annotation/brenda-tissue-ontology.obo')

Read obo graph
Name: 
Type: DiGraph
Number of nodes: 5659
Number of edges: 6430
Average in degree:   1.1362
Average out degree:   1.1362


In [12]:
root_id = 'BTO:0000089'

In [19]:
import pandas as pd

In [68]:
api.samples_count(tissue_id=[root_id]+to.descendants(root_id), norm=1, age_exists=1)

2624

In [66]:
c = dict(tissue_id=[root_id]+to.descendants(root_id), norm=1, age_exists=1)
Api.conds_to_mongo(c)

{'annotation.age': {'$ne': None},
 'annotation.norm': 1,
 'annotation.tissue_id': {'$in': ['BTO:0000089',
   'BTO:0000686',
   'BTO:0004053',
   'BTO:0000381',
   'BTO:0000237',
   'BTO:0000131',
   'BTO:0000574',
   'BTO:0000125',
   'BTO:0000553',
   'BTO:0000102']}}

In [ ]:
# api.search_samples(tissue_id=[root_id]+to.descendants(root_id), norm=1, age_exists=1)

In [173]:
b = (
    pd.DataFrame(list(api.search_samples(tissue_id=[root_id]+to.descendants(root_id), norm=1, age_exists=1)))
    [['accession', 'annotation', 'platform', 'series']].copy()
)

In [174]:
b.shape


(2624, 4)

In [175]:
b['tissue_id'] = b['annotation'].map(lambda a: a['tissue_id'])
b['norm'] = b['annotation'].map(lambda a: a.get('norm', 0))
b['preprocessed'] = b['annotation'].map(lambda a: a.get('preprocessed', 0))
b['age'] = b['annotation'].map(lambda a: a.get('age', -1))

In [168]:
b.head()

,accession,annotation,platform,series,tissue_id,norm,preprocessed,age
0,GSM1682703,"{'age': '23', 'tissue_id': 'BTO:0000089', 'nor...",GPL570,[GSE68833],BTO:0000089,1,0,23
1,GSM1686337_2,"{'age': '39', 'tissue_id': 'BTO:0000089', 'nor...",GPL20163,"[GSE68908, GSE68909]",BTO:0000089,1,0,39
2,GSM1686338_2,"{'age': '50', 'tissue_id': 'BTO:0000089', 'nor...",GPL20163,"[GSE68908, GSE68909]",BTO:0000089,1,0,50
3,GSM1686339_2,"{'age': '49', 'tissue_id': 'BTO:0000089', 'nor...",GPL20163,"[GSE68908, GSE68909]",BTO:0000089,1,0,49
4,GSM1686340_2,"{'age': '36', 'tissue_id': 'BTO:0000089', 'nor...",GPL20163,"[GSE68908, GSE68909]",BTO:0000089,1,0,36


In [77]:
b.norm.value_counts()

1    2624
dtype: int64

In [78]:
b.preprocessed.value_counts()

0    2140
1     484
dtype: int64

In [81]:
blood_platforms = b.platform.unique().tolist()

In [83]:
import pymongo
db = pymongo.MongoClient().scraper_test_dev

In [117]:
blood_platforms_df = pd.DataFrame(list(db.platforms.find({'accession': {'$in': blood_platforms}}))).drop_duplicates(subset=['accession'])
blood_platforms_df.head()

,_id,accession,alternatives,catalog_number,coating,contact,data_source,description,distribution,last_update_date,...,organism,publication_date,scrap_date,status,submission_date,supplementary_files,support,technology,title,web_link
0,561bedb1a1385773a41fd931,A-AFFY-142,[],None,unknown_surface_type,"[Affymetrix, Inc. (support@affymetrix.com)]",array-express,The Affymetrix Genome-Wide Human SNP Array 6.0...,None,None,...,None,NaN,NaN,None,None,[{'url': 'http://ftp.ebi.ac.uk/pub/databases/m...,None,None,Affymetrix GeneChip Genome-Wide Human SNP 6.0 ...,None
37,561bedb1a1385773a41fd913,A-AFFY-184,[],None,None,[Affymetrix (support@affymetrix.com)],array-express,"""#%create_date=Wed Dec 22 13:03:44 PST 2010#%g...",None,None,...,Acyrthosiphon pisum;Saguinus labiatus;Nasonia ...,NaN,NaN,None,None,[{'url': 'http://ftp.ebi.ac.uk/pub/databases/m...,None,None,Affymetrix GeneChip miRNA 2.0 Array [miRNA-2_0],None
49,561bedb1a1385773a41fd93d,A-AFFY-33,[],None,glass,"[Affymetrix, Inc. (support@affymetrix.com)]",array-express,None,None,None,...,None,NaN,NaN,None,None,[{'url': 'http://ftp.ebi.ac.uk/pub/databases/m...,None,None,Affymetrix GeneChip Human Genome HG-U133A [HG-...,None
206,561bedb1a1385773a41fd8f9,A-AFFY-44,[],None,glass,"[Affymetrix, Inc. (support@affymetrix.com)]",array-express,None,None,None,...,Homo sapiens,NaN,NaN,None,None,[{'url': 'http://ftp.ebi.ac.uk/pub/databases/m...,None,None,Affymetrix GeneChip Human Genome U133 Plus 2.0...,None
607,561bedaca1385773a41fd8dc,A-AGIL-28,[],None,None,"[""Agilent Technologies, Inc (Americas) (dna_mi...",array-express,Whole Human Genome Microarray 4x44K. Agilent-0...,None,None,...,Homo sapiens,NaN,NaN,None,None,[{'url': 'http://ftp.ebi.ac.uk/pub/databases/m...,None,None,Agilent Whole Human Genome Microarray 4x44K 01...,None


In [89]:
from lib.utils import expand_dataframe

In [119]:
blood_platforms_df['manufaturer_assigned'] = blood_platforms_df.manufaturer_assigned.fillna('unknown')

In [120]:
blood_platforms_df[['accession', 'title', 'manufacturer', 'manufaturer_assigned']].head()

,accession,title,manufacturer,manufaturer_assigned
0,A-AFFY-142,Affymetrix GeneChip Genome-Wide Human SNP 6.0 ...,None,affymetrix
37,A-AFFY-184,Affymetrix GeneChip miRNA 2.0 Array [miRNA-2_0],None,affymetrix
49,A-AFFY-33,Affymetrix GeneChip Human Genome HG-U133A [HG-...,None,affymetrix
206,A-AFFY-44,Affymetrix GeneChip Human Genome U133 Plus 2.0...,None,affymetrix
607,A-AGIL-28,Agilent Whole Human Genome Microarray 4x44K 01...,None,agilent


In [169]:
stats = (
    expand_dataframe(b, 'series')
    .groupby(['platform', 'series', 'preprocessed'])
    .size()
    .to_frame()
    .reset_index()
    .rename(columns={0: 'count'})
    .merge(blood_platforms_df[['accession', 'title', 'manufacturer', 'manufaturer_assigned']], left_on='platform', right_on='accession', how='left')
    .drop('manufacturer', axis=1)
    .drop('accession', axis=1)
#     .query('platform=="GPL6947"')
#     .sort('series')
)
stats.head()

,platform,series,preprocessed,count,title,manufaturer_assigned
0,A-AFFY-142,E-MTAB-1051,0,2,Affymetrix GeneChip Genome-Wide Human SNP 6.0 ...,affymetrix
1,A-AFFY-142,E-MTAB-1159,0,2,Affymetrix GeneChip Genome-Wide Human SNP 6.0 ...,affymetrix
2,A-AFFY-184,E-MTAB-2785,0,27,Affymetrix GeneChip miRNA 2.0 Array [miRNA-2_0],affymetrix
3,A-AFFY-33,E-MEXP-72,0,1,Affymetrix GeneChip Human Genome HG-U133A [HG-...,affymetrix
4,A-AGIL-28,E-TABM-595,0,17,Agilent Whole Human Genome Microarray 4x44K 01...,agilent


In [157]:
(
    stats
    .query('preprocessed == 1')
)

,platform,series,preprocessed,count,title,manufaturer_assigned
49,GPL570,GSE11375,1,17,[HG-U133_Plus_2] Affymetrix Human Genome U133 ...,affymetrix
50,GPL570,GSE14845,1,1,[HG-U133_Plus_2] Affymetrix Human Genome U133 ...,affymetrix
51,GPL570,GSE18781,1,25,[HG-U133_Plus_2] Affymetrix Human Genome U133 ...,affymetrix
52,GPL570,GSE39088,1,64,[HG-U133_Plus_2] Affymetrix Human Genome U133 ...,affymetrix
53,GPL570,GSE54837,1,90,[HG-U133_Plus_2] Affymetrix Human Genome U133 ...,affymetrix
55,GPL571,GSE46097,1,189,[HG-U133A_2] Affymetrix Human Genome U133A 2.0...,affymetrix
56,GPL571,GSE66175,1,189,[HG-U133A_2] Affymetrix Human Genome U133A 2.0...,affymetrix
60,GPL6244,GSE27131,1,7,[HuGene-1_0-st] Affymetrix Human Gene 1.0 ST A...,affymetrix
61,GPL6244,GSE50635,1,16,[HuGene-1_0-st] Affymetrix Human Gene 1.0 ST A...,affymetrix
76,GPL6947,GSE29147,1,2,Illumina HumanHT-12 V3.0 expression beadchip,illumina


In [131]:
(
    stats
    .query('preprocessed == 0 and platform == "GPL570"')
)

,platform,series,preprocessed,count,accession,title,manufaturer_assigned
54,GPL570,GSE68833,0,1,GPL570,[HG-U133_Plus_2] Affymetrix Human Genome U133 ...,affymetrix


In [161]:
(
    stats
    [stats.title.map(lambda t: 'methyl' not in (t or '').lower())]
    .query('manufaturer_assigned == "illumina"')
        .query('count >= 5')
    .sort('count', ascending=False)

    
)

,platform,series,preprocessed,count,title,manufaturer_assigned
23,GPL10558,GSE61672,0,336,Illumina HumanHT-12 V4.0 expression beadchip,illumina
80,GPL6947,GSE46743,0,182,Illumina HumanHT-12 V3.0 expression beadchip,illumina
79,GPL6947,GSE38485,1,96,Illumina HumanHT-12 V3.0 expression beadchip,illumina
78,GPL6947,GSE38484,1,96,Illumina HumanHT-12 V3.0 expression beadchip,illumina
74,GPL6947,GSE19491,0,81,Illumina HumanHT-12 V3.0 expression beadchip,illumina
75,GPL6947,GSE22098,0,81,Illumina HumanHT-12 V3.0 expression beadchip,illumina
82,GPL6947,GSE51406,0,24,Illumina HumanHT-12 V3.0 expression beadchip,illumina
81,GPL6947,GSE51405,0,24,Illumina HumanHT-12 V3.0 expression beadchip,illumina
26,GPL11154,GSE46579,0,22,Illumina HiSeq 2000 (Homo sapiens),illumina
71,GPL6883,GSE38481,0,22,Illumina HumanRef-8 v3.0 expression beadchip,illumina


In [160]:
(
    stats
    .query('manufaturer_assigned == "agilent"')
        .query('count >= 5')
    .sort('count', ascending=False)

#     .platform
#     .value_counts()
)

,platform,series,preprocessed,count,title,manufaturer_assigned
70,GPL6848,GSE19738,0,66,Agilent-012391 Whole Human Genome Oligo Microa...,agilent
62,GPL6480,GSE33566,0,28,Agilent-014850 Whole Human Genome Microarray 4...,agilent
63,GPL6480,GSE48424,0,18,Agilent-014850 Whole Human Genome Microarray 4...,agilent
6,A-AGIL-28,E-TABM-595,0,17,Agilent Whole Human Genome Microarray 4x44K 01...,agilent
64,GPL6480,GSE51216,0,15,Agilent-014850 Whole Human Genome Microarray 4...,agilent
40,GPL17077,GSE68906,0,12,Agilent-039494 SurePrint G3 Human GE v2 8x60K ...,agilent
41,GPL17077,GSE68909,0,12,Agilent-039494 SurePrint G3 Human GE v2 8x60K ...,agilent
45,GPL20163,GSE68908,0,12,Agilent-067877 Homo sapiens hg19 Genocheck_CH3...,agilent
46,GPL20163,GSE68909,0,12,Agilent-067877 Homo sapiens hg19 Genocheck_CH3...,agilent
14,A-MEXP-2276,E-MTAB-1467,0,8,Agilent 031181 Human and human viral miRNAs 8x60k,agilent


In [172]:
(
    stats
    .query('manufaturer_assigned == "agilent"')
        .query('count >= 5')
    .sort('count', ascending=False)

    .platform
    .unique()
)

array(['GPL6848', 'GPL6480', 'A-AGIL-28', 'GPL17077', 'GPL20163',
       'A-MEXP-2276', 'GPL16770', 'A-MEXP-2140'], dtype=object)

In [145]:
FRMA_PLATFORMS = ['GPL' + str(x) for x in [
    570, 571, 96,
    1261,  # organism Mus musculus
    6244,  # getting strange errors. data in matrix
    # 1355,  # NO FRMA, Rattus norvegicus,
    6246,  # no frma package (?). read.affybatch: The affy package can process data from the Gene ST 1.x series of arrays,
    # but you should consider using either the oligo or xps packages, which are specifically
    # designed for these arrays.
    # there is no package called ‘mogene10stv1frmavecs’

    90, # organism Saccharomyces cerevisiae
    3921,
    # 5175, # no frma package (?). read.affybatch: The affy package is not designed for this array type. 
    # Please use either the oligo or xps package.
    # 5188, # no cdf frma packages
    8321,  # organism Mus musculus
]]


In [170]:
(
    stats
    .assign(frma=lambda d: d.platform.isin(FRMA_PLATFORMS).astype(int))
    .query('manufaturer_assigned == "affymetrix"')
    .query('count >= 5')
    .sort('count', ascending=False)
#     .platform
#     .value_counts()
)

,platform,series,preprocessed,count,title,manufaturer_assigned,frma
55,GPL571,GSE46097,1,189,[HG-U133A_2] Affymetrix Human Genome U133A 2.0...,affymetrix,1
56,GPL571,GSE66175,1,189,[HG-U133A_2] Affymetrix Human Genome U133A 2.0...,affymetrix,1
47,GPL570,E-MTAB-1532,0,100,[HG-U133_Plus_2] Affymetrix Human Genome U133 ...,affymetrix,1
53,GPL570,GSE54837,1,90,[HG-U133_Plus_2] Affymetrix Human Genome U133 ...,affymetrix,1
52,GPL570,GSE39088,1,64,[HG-U133_Plus_2] Affymetrix Human Genome U133 ...,affymetrix,1
2,A-AFFY-184,E-MTAB-2785,0,27,Affymetrix GeneChip miRNA 2.0 Array [miRNA-2_0],affymetrix,0
51,GPL570,GSE18781,1,25,[HG-U133_Plus_2] Affymetrix Human Genome U133 ...,affymetrix,1
84,GPL8786,GSE46729,0,24,[miRNA-1_0] Affymetrix miRNA Array,affymetrix,0
49,GPL570,GSE11375,1,17,[HG-U133_Plus_2] Affymetrix Human Genome U133 ...,affymetrix,1
48,GPL570,E-TABM-666,0,16,[HG-U133_Plus_2] Affymetrix Human Genome U133 ...,affymetrix,1


In [171]:
(
    stats
    .assign(frma=lambda d: d.platform.isin(FRMA_PLATFORMS).astype(int))
    .query('manufaturer_assigned == "unknown"')
    .query('count >= 5')
    .sort('count', ascending=False)
#     .platform
#     .value_counts()
)

,platform,series,preprocessed,count,title,manufaturer_assigned,frma
13,A-MEXP-2396,E-MTAB-2564,0,58,Scilifelab-KTH_BIONMD_AntibodyBeadArray_384,unknown,0
6,A-MEXP-192,E-TABM-10,0,30,Breakthrough human CGH 4.6k 1.1.2,unknown,0
65,GPL6782,GSE23869,0,22,MHP Human 1Mb-resolution CGH array,unknown,0
33,GPL15497,GSE50013,0,20,Applied Biosystems TaqMan Array Human MicroRNA...,unknown,0
29,GPL13618,GSE29707,0,17,IPA/Center of Molecular Medicine Human_Mouse_R...,unknown,0
11,A-MEXP-2220,E-MEXP-3647,0,16,CENAT-H.sapiens-AML-dedicated,unknown,0
25,GPL11241,GSE55515,0,15,"miRCURY LNA microRNA Array, 5th generation - h...",unknown,0
10,A-MEXP-2209,E-MEXP-3646,0,14,None,unknown,0
9,A-MEXP-2209,E-MEXP-3633,0,14,None,unknown,0
14,A-MEXP-2399,E-MTAB-2494,0,12,WY.human.KBD screening chip.v01,unknown,0
